In [1]:
import numpy as np
import sys,sklearn
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'AppleGothic'

In [2]:
df = pd.read_csv('NHIS_OPEN_GJ_2018_100.csv')

In [3]:
df_s = df[['성별코드','신장(5Cm단위)', '체중(5Kg단위)', '연령대코드(5세단위)','이완기혈압','수축기혈압','식전혈당(공복혈당)', '허리둘레']]

In [4]:
df_s = df_s.rename(columns={"연령대코드(5세단위)":"연령대코드",
                            "체중(5Kg단위)":"체중",
                            "신장(5Cm단위)":"신장",             
                            "식전혈당(공복혈당)":"식전혈당"})

In [5]:
df_s.head(5)

,성별코드,신장,체중,연령대코드,이완기혈압,수축기혈압,식전혈당,허리둘레
0,2,160,60,7,60.0,110.0,96.0,79.5
1,1,170,55,6,78.0,128.0,79.0,69.3
2,1,165,70,12,65.0,128.0,80.0,85.0
3,2,150,45,15,89.0,151.0,100.0,71.5
4,2,145,50,14,62.0,114.0,124.0,77.0


## 허리둘레(40이상 140 이하) 

In [6]:
df_s = df_s.loc[(df_s['허리둘레'] <= 140) & (df_s['허리둘레'] >= 40)]

In [7]:
#diabete 식전혈당 (127이상:2, 100이상126미만:1, 정상 : 0)
def diabete (x) : 
    if (x >= 127) : return 2
    elif (x>=100 and x <= 126 ): return 1
    else : return 0
#bmi 단계  (0:정상, 1:과체중, 2:비만)
def bmi(h,w):
    bmi = w / ((h/100)**2)
    if (bmi >= 30) : return 2
    elif (bmi < 29 and bmi >= 25) : return 1
    else : return 0
    
#혈압 0:정상 , 1:전단계, 2:1기, 3:2기고혈압기고혈압  
def pulse(l,h):
    if (l >= 100 or h >= 160):
        return 3
    elif (90 <= l < 100) or (140 <= h < 160):
        return 2
    elif (80 <= l < 90) or (120 <= h < 140):
        return 1
    else:
        return 0
    
def bokbu(sex, waist) :
    if (sex == 1) and (waist >= 90) :
        return 1
    elif (sex == 2) and (waist >= 85) :
        return 2
    else : return 0
    

In [8]:
# 엄세진님 코드 참고했습니다.
df_s['혈당'] = df_s['식전혈당'].map(diabete)
df_s['고혈압'] = np.vectorize(pulse)(df_s['이완기혈압'], df_s['수축기혈압'])
df_s['bmi'] = np.vectorize(bmi)(df_s['신장'], df_s['체중'])
df_s['복부비만'] = np.vectorize(bokbu)(df_s['성별코드'], df_s['허리둘레'])
df_s.describe()

D:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2192: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


,성별코드,신장,체중,연령대코드,이완기혈압,수축기혈압,식전혈당,허리둘레,혈당,고혈압,bmi,복부비만
count,999566.000000,999566.000000,999566.000000,999566.000000,993848.000000,993848.000000,993658.000000,999566.000000,999566.000000,999566.000000,999566.000000,999566.000000
mean,1.464198,162.293931,63.591399,10.597882,76.078154,122.674750,100.874225,81.451995,0.461249,0.767358,0.350699,0.336663
std,0.498717,9.336116,12.730774,2.878849,10.010737,14.623355,24.111770,9.676727,0.632140,0.751803,0.572946,0.638203
min,1.000000,125.000000,30.000000,5.000000,27.000000,63.000000,23.000000,40.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,155.000000,55.000000,8.000000,70.000000,112.000000,89.000000,75.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,160.000000,60.000000,11.000000,76.000000,121.000000,96.000000,81.300000,0.000000,1.000000,0.000000,0.000000
75%,2.000000,170.000000,70.000000,13.000000,82.000000,131.000000,105.000000,88.000000,1.000000,1.000000,1.000000,0.000000
max,2.000000,190.000000,135.000000,18.000000,173.000000,267.000000,872.000000,140.000000,2.000000,3.000000,2.000000,2.000000


# 엔트로피

In [9]:
def Entropy(df, attribute) :
    list1  = list(df[attribute].unique())
    etp = 0
    for i in range(len(list1)):
        q1 = len(df[df[attribute] == list1[i]])/len(df[attribute])
        etp -=  q1*np.log2(q1)
    return etp

def Group_info(df,m_attribute,fix_attribute):
    list2 = list(df[m_attribute].unique())
    info_Entropy = Entropy(df,fix_attribute)
    info_data = 0
    for i in range (len(list2)) :
        q2 = df.loc[df[m_attribute] == list2[i]]
        info_data +=  (len(q2[m_attribute])/len(df[m_attribute]))* Entropy(q2,fix_attribute) 
    
    data = info_Entropy - info_data
    
    return data

info = np.array([("성별코드"),("고혈압"),("bmi"),("복부비만"),("연령대코드")])

for i in info:
    print(i, ":", Group_info(df_s, i, "혈당") * 1000)

성별코드 : 14.752319083181753
고혈압 : 30.46176497542774
bmi : 15.803809581364625
복부비만 : 21.462026403340495
연령대코드 : 45.842340350574105


In [10]:
Entropy(df_s,"혈당")

1.2372064525910793

### 지니 지수 https://lucy-the-marketer.kr/ko/growth/decision-tree-and-impurity/

In [11]:
def Gini(df, attribute) :
    list1  = list(df[attribute].unique())
    gini = 0
    for i in range(len(list1)):
        q1 = len(df[df[attribute] == list1[i]])/len(df[attribute])
        gini += (q1**2)
    return 1 - gini

def Group_Gini(df,m_attribute,fix_attribute):
    list2 = list(df[m_attribute].unique())
    info_data = 0
    for i in range (len(list2)) :
        q2 = df.loc[df[m_attribute] == list2[i]]
        info_data +=  (len(q2[m_attribute])/len(df[m_attribute]))* Gini(q2,fix_attribute) 
    
    return info_data

info = np.array([("성별코드"),("고혈압"),("bmi"),("복부비만"),("연령대코드")])

for i in info :
    print(i,":",Group_Gini(df_s, i,"혈당")*100)

성별코드 : 51.332654007454
고혈압 : 50.61087493173404
bmi : 51.30041746755184
복부비만 : 51.10639071882977
연령대코드 : 50.2365301025295
